In [1]:
#NOTEBOOK 2: implementazione della CV per una stima più accurata delle performance del classificatore e di una versione
#di pseudo-labeling in approccio semi-supervised che accetti solo labels certificate

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#lettura del file excel prodotto dal Notebook 1 in pandas

import pandas as pd
!pip install --upgrade openpyxl

data = pd.read_excel('/content/drive/MyDrive/df_preprocessed.xlsx')
data = data.drop('Unnamed: 0', axis=1)
#print(data.head)
print(data.columns)
print(data.shape)

#totale righe: 5877

Index(['Descrizione', 'Numero', 'PrezzoUnitario', 'PrezzoTotale',
       'AliquotaIVA', 'Conto', 'Year', 'Month', 'Day', 'CodiceTipo_',
       ...
       'yes', 'yoga', 'yogurt', 'yonkers', 'zacapa', 'zenzero', 'zero',
       'zuccheriera', 'zucchero', 'zucher'],
      dtype='object', length=1564)
(5877, 1564)


In [3]:
#eliminazione delle righe ausiliarie

mask = (data.Descrizione != 'Riga ausiliaria contenente informazioni tecniche e aggiuntive del documento')
data = data[mask]
data.reset_index(inplace=True)
data = data.drop('index', axis=1)

#print(data)

#rimangono 5692 righe

In [4]:
#in vista del semi supervised learning, separazione dei dati supervisionati e non supervisionati

from sklearn.semi_supervised import SelfTrainingClassifier

data_labeled = data[data['Conto'].notna()]
data_unlabeled = data[data['Conto'].isnull()]

data_labeled.reset_index(inplace=True)
data_labeled = data_labeled.drop('index', axis=1)
data_unlabeled.reset_index(inplace=True)
data_unlabeled = data_unlabeled.drop('index', axis=1)

print(data_labeled.shape)
print(data_unlabeled.shape)

(1805, 1564)
(3887, 1564)


In [7]:
#TOPIC 1: introduzione anche della validazione fatta per CV. Con questa tecnica è possibile operare una grid
#search nello spazio degli iperparametri in fase di validazione senza ridurre il training set ma anche avere una stima
#più accurata della metrica di valutazione. 
#Inoltre, er avere un risultato paragonabile al caso base nel Notebook 1 quando testerò il modello validato, uso lo stesso
#training e lo stesso test set

mask = (data_labeled['Conto'] != '18/40/501')&(data_labeled['Conto'] != '66/25/505')&(data_labeled['Conto'] != '66/25/508')&(data_labeled['Conto'] != '66/30/060')&(data_labeled['Conto'] != '68/05/005')&(data_labeled['Conto'] != '68/05/133')&(data_labeled['Conto'] != '68/05/290')&(data_labeled['Conto'] != '68/05/320')&(data_labeled['Conto'] != '68/05/385')&(data_labeled['Conto'] != '68/05/407')&(data_labeled['Conto'] != '88/20/035')
data_labeled2 = data_labeled[mask]
data_labeled2.reset_index(inplace=True)
data_labeled2 = data_labeled2.drop('index', axis=1)

data_test = data_labeled2.loc[1249:,:]
#print(data_test)

data_train = data_labeled2.loc[:1248,:]
#print(data_train)

#quando elimino i conti rari però non sovrascrivo data_labeled che serve integro nella successiva applicazione del semi
#supervised learning

In [8]:
#Osservato che accuracy e F1 score danno indicazioni simili (nel Notebook 1) si procede con l'accuracy che è più interpretabile; mentre per
#quanto riguarda i modelli si procede con RF e AdaBoost.
#La tecnica di CV scelta è StratifiedGroupKFold a causa dello sbilanciamento delle classi e della presenza di gruppi fra
#i samples (i.e. righe fattura), cioè le fatture stesse identificate dai numeri di fattura.

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedKFold
import numpy as np

X = data_train.drop(['Conto', 'Numero', 'Descrizione'], axis = 1)
y = data_train.Conto
groups = data_train.Numero

cv = StratifiedGroupKFold(n_splits=5)

In [9]:
#stima delle metriche tramite 5-fold CV

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree


#clf = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=8), n_estimators=100)
clf = RandomForestClassifier(n_estimators=100, max_depth=30, bootstrap = False)
scores1 = cross_val_score(clf, X, y, cv=cv, scoring='accuracy', groups=groups)
scores2 = cross_val_score(clf, X, y, cv=cv, scoring='f1_weighted', groups=groups)
scores3 = cross_val_score(clf, X, y, cv=cv, scoring='f1_macro', groups=groups)
print(scores1)
print(scores2)
print(scores3)

#anche le stime per CV confermano sostanzialmente quelle fatte per hold-out: accuracy e F1 weighted a 0.94/0.95, se si ignora
#il primo fold. F1 macro è più bassa a causa di alcune piccole classi non predette correttamente nella divisione in folds.

[0.57594937 0.97407407 0.9123506  0.93658537 0.93236715]
[0.52555567 0.96244816 0.9175626  0.91858397 0.92314966]
[0.72033757 0.80311284 0.68525957 0.84243789 0.87706362]


In [ ]:
#analisi del perchè (a prescindere dal numero di folds scelto) il primo score è bassissimo.

unique, counts = np.unique(groups, return_counts=True)
print(unique)
print(counts)

#c'è un gruppo (il numero fattura è 1200261973) numerosissimo con 125 istanze e tutte hanno label 66/25/005,
#perciò quando a questo gruppo tocca fare da test l'algoritmo non impara a riconoscere tale classe e sbaglia la predizione
#su gran parte degli elementi abbassando drasticamente la metrica.

for train_idxs, test_idxs in cv.split(X, y, groups):
  print(np.unique(groups[train_idxs]))
  print(np.unique(groups[test_idxs]))

#come prevedibile, il gruppo con numero di fattura 1200261973 è nel test al primo split, per questo le metriche associate 
#sono così basse. Tali gruppi così sbilanciati rendono impossibile un'adeguata stratificazione, ma comunque StratifiedGroupKFold
#lavora meglio di GroupKFold

In [17]:
#ricerca dei migliori iper-parametri (numero di alberi e profondità) per RF grazie alla CV

from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators': (25, 50, 100, 200, 500), 'max_depth': (5, 8, 10, 12, 15, 20, 25, 30, 50, 100, 200)}
rf = RandomForestClassifier(bootstrap = False)
clf = GridSearchCV(rf, parameters, cv=cv, scoring='accuracy')
clf.fit(X, y, groups=groups)

print(clf.best_params_)

#risulta ottimale usare alberi abbastanza profondi (learners overfittati tanto la varianza viene abbassata dall'averaging siccome RF è un
#metodo di bagging), ma comunque da profondità 25 circa in poi non fa grande differenza, come evidente dalla cella sotto

{'max_depth': 30, 'n_estimators': 100}


In [ ]:
#performance su tutte le possibili coppie di iper parametri in termini di accuracy. I valori non sono intorno a 0.94, come
#nelle stime con hold-out, per via del primo fold di CV su cui performa male

means = clf.cv_results_["mean_test_score"]
stds = clf.cv_results_["std_test_score"]
for mean, std, params in zip(means, stds, clf.cv_results_["params"]):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))


In [19]:
#test sul solito test set finora tenuto in disparte

y_pred = clf.predict(data_test.drop(['Conto', 'Numero', 'Descrizione'], axis = 1))
print(accuracy_score(data_test.Conto, y_pred))

#dato che la grid search non ha evidenziato coppie di iper-paramentri molto migliori di quelli usati a occhio precedentemente
#non c'è da stupirsi che l'accuracy sul test sia intorno a 0.94/0.95

0.9531835205992509


In [ ]:
#come boosting method Gradient Boosting invece che AdaBoost perchè quest'ultimo prende il modello da boostare
#come iper-paramentro e non la profondità dell'albero. Per il resto la procedura è uguale a quella sopra per RF eccetto
#che si usano alberi meno profondi perchè per i metodi di boosting sono richiesti weak learners e la griglia degli
#iper-parametri è tridimensionale

from sklearn.ensemble import GradientBoostingClassifier


parameters = {'n_estimators': (50, 100, 200), 'max_depth': (3, 4, 5), 'learning_rate': (0.1, 1.0)}
gbc = GradientBoostingClassifier()
clf = GridSearchCV(gbc, parameters, cv=cv, scoring='accuracy')
clf.fit(X, y, groups=groups)

print(clf.best_params_)

#il fitting richiede molto più tempo di RF e come prevedibile è ottimizzato da learners poco profondi

In [ ]:
y_pred = clf.predict(data_test.drop(['Conto', 'Numero', 'Descrizione'], axis = 1))
print(accuracy_score(data_test.Conto, y_pred))

#anche per questo ensemble method si ottengono performance paragonabili a RF e AdaBoost, forse leggermente peggiore
#(potrebbe essere dovuto al fatto che la grid search è stata fatta piu ristretta)

0.9344569288389513


In [5]:
#TOPIC 2: implementazione del semi supervised learning con validazione delle pseudo-label grazie agli
#ammontari associati a ciascun conto in prima nota. Per prima cosa viene caricato il modello di classificazione
#RF trainato su tutti i dati nel notebook 1(senza dividere tra training e test set e senza escludere i samples con conti rari, così
#da massimizzare la probabilità di assegnare correttamente una pseudo-label)

import pickle

clf = pickle.load(open('/content/drive/MyDrive/finalized_model.sav', 'rb'))
print(clf)

#in seguito alle considerazioni fatte con la grid search per CV è un RF con 100 alberi di profondità massima 30

RandomForestClassifier(bootstrap=False, max_depth=30)


In [7]:
#selezione, tra i samples senza conto a cui si punta ad associare una pseudo-label, dei soli matchabili ovvero
#quelli aventi, innanzitutto, un corrispondente in prima nota

import numpy as np

prima_nota = pd.read_excel('/content/df_row.xlsx')
prima_nota = prima_nota.drop('Unnamed: 0', axis=1)

numeri_fattura_registrati = np.unique(prima_nota['ND ori.'])
mask = []
for elem in data_unlabeled.Numero:
  mask.append(elem in numeri_fattura_registrati)

data_unlabeled2 = data_unlabeled[mask]
data_unlabeled2.reset_index(inplace=True)
data_unlabeled2 = data_unlabeled2.drop('index', axis=1)

print(data_unlabeled2.shape)

#su 3887 dati unsupervised, solo 1297 di essi hanno il corrispondente in prima nota

(1297, 1564)


In [8]:
#bisogna però osservare che, in alcuni casi, pur essendoci la corrispondente registrazione in prima nota, gli ammontari non
#coincidono, probabilmente a causa di qualche errore. Ricerca di questi casi (usando una soglia di 0.02 con l'idea che un errore
#maggiore di questo non è attribuibile ad approssimazioni)

all_fatture = list(np.unique(data_unlabeled2.Numero))

non_corresponding = []

for numero_fattura in all_fatture:
  ammontare_righe_fattura = np.sum(data_unlabeled2.PrezzoTotale[(data_unlabeled2['Numero'] == numero_fattura)])
  ammontare_prima_nota = np.sum((prima_nota.Importo[(prima_nota['ND ori.'] == numero_fattura)].values)[2:])
  if np.abs(ammontare_righe_fattura - ammontare_prima_nota) > 0.02:
    non_corresponding.append(numero_fattura)

print(non_corresponding)
print(len(non_corresponding))

#I motivi di incongruenza delle 31 fatture non_corresponding possono essere vari. Alcuni esempi a seguire.
#numeri strani: 423, in prima nota sono registrate tutte nel conto "vino", ma palesemente dalla descrizione non si tratta
#di vino, inoltre l'ammontare è 89.38 ma in prima nota è segnato 120. 1101, l'ammontare è 210 ma in prima nota è segnato
#62.64. 112, anche qua non coincidono gli ammontari, inoltre, l'algoritmo assegna giustamente il conto "vino" (si vede dalla descrizione)
#mentre in prima nota queste spese sono registrate come "materie di consumo", infine, essendoci un solo conto sarebbe risolvibile
#il knapsack ma dato che gli ammontari non tornano non trova nessuna soluzione ammissibile. 119, stesso problema. 2253/C, 
#non tutte le righe fattura sono registrate in prima nota (solo 2 su 4), perciò ovviamente gli ammontari non tornano.
#12/1040, sebbene abbia solo 5 righe e 2 conti non viene risolto dal knapsack perchè gli ammontari in fattura e prima nota differiscono
#di 0.01 mentre nella funzione di match si è usata come soglia max 0.001. Simile per 2069036018. Queste comunque non
#risultano nelle non_corresponding a causa della soglia 0.02 usata sopra, dato che sono recuperabili con A* con la soglia opportuna.

['1042', '1101', '112', '119', '131', '1402', '150', '166', '17', '202022547923', '202022602551', '202022656839', '2020FTA.20.003470', '2253/C', '25', '2635', '27', '3', '31', '34', '4', '41', '423', '57', '58', '59440', '635', '7', '9', '90', 'FPR 127/20']
31


In [9]:
count = 0
for elem in list(data_unlabeled2.Numero):
  if elem in non_corresponding:
    count = count + 1

print(count)

#altre 141 righe non sono matchabili a priori perchè, pur essendoci la registrazione in prima nota, gli ammontari non tornano

141


In [10]:
#rimuovo le corrispondenti fatture da data_unlabeled2, per lo stesso motivo per cui avevo rimosso quelle senza corrispondente 
#in prima nota

mask = []
for elem in data_unlabeled2.Numero:
  mask.append(elem not in non_corresponding)

data_unlabeled3 = data_unlabeled2[mask]
data_unlabeled3.reset_index(inplace=True)
data_unlabeled3 = data_unlabeled3.drop('index', axis=1)

print(data_unlabeled3.shape)

#dunque sono 1156 le righe fattura potenzialmente recuperabili con le pseudo-labels perchè solo per queste c'è un'affidabile registrazione
#in prima nota per validare le pseudo-label

(1156, 1564)


In [11]:
#inoltre, solo se i conti della registrazione in prima nota sono conti noti nel training il nostro metodo di pseudo-labeling
#può funzionare. Per trovare i conti noti si osserva data_labeled e non data_labeled2 in cui i conti poco comuni sono stati eliminati

conti_noti = np.unique(data_labeled.Conto)
mask2 = []
for numero_fattura in data_unlabeled3.Numero:
  boolean = True
  conti_prima_nota = prima_nota.Conto[(prima_nota['ND ori.'] == numero_fattura)]
  for conto in conti_prima_nota[2:]:
    if not (conto in conti_noti):
      boolean = False
  mask2.append(boolean)

print(mask2)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, Tru

In [12]:
#analisi dei samples rimasti

data_unlabeled3 = data_unlabeled3[mask2]
data_unlabeled3.reset_index(inplace=True)
data_unlabeled3 = data_unlabeled3.drop('index', axis=1)

print(data_unlabeled3.shape)

#delle 1156 rimangono 1049 samples potenzialmente pseudo-etichetabili; gli altri hanno delle registrazioni in prima nota
#mai viste nel training set dal classificatore

(1049, 1564)


In [13]:
#classificazione delle 1049 righe rimaste tramite il classificatore RF trainato su tutti i dati noti

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

features = data_unlabeled3.drop(['Conto', 'Descrizione', 'Numero'], axis=1)
descr = data_unlabeled3.Descrizione
num = data_unlabeled3.Numero

y_pred = clf.predict(features)
print(len(y_pred))

1049


In [15]:
#creazione del file con le pseudo-labels da trattare con A*

pseudo_labels = pd.concat(objs=(data_unlabeled3.Numero, pd.Series(y_pred, name='Conto'), data_unlabeled3.PrezzoTotale), axis=1)
print(pseudo_labels.shape)
print(pseudo_labels.columns)

pseudo_labels.to_excel('/content/pseudo_labels2.xlsx')

(1049, 3)
Index(['Numero', 'Conto', 'PrezzoTotale'], dtype='object')


In [16]:
#IN REALTA' DA QUANDO HO L'ALGORITMO A* SU PYCHARM LE PROSSIME 3 CELLE SONO SUPERFLUE PERCHE' QUESTA VALIDAZIONE NON E' 
#ALTRO CHE IL RAGGIUNGIMENTO DEL GOL CON A* IN 0 PASSI

#per la validazione bisogna fare un controllo incrociato con i valori registrati in prima nota per ogni fattura e per ogni
#conto: se tutti i check corrispondono allora si possono validare tutte le predizioni effettutate sulle righe di tale fattura.
#Per procedere si cicla su entrambi gli indici dell'oggetto raggruppato; ad ogni iterazione idx contiene i 2 indici gerarchici
#e data il valore numerico associato a tale registrazione.

df = pd.concat(objs=[data_unlabeled3.Numero, data_unlabeled3.Descrizione, pd.Series(y_pred).rename('Conto'), data_unlabeled3.PrezzoTotale], axis=1)
grouped = df.groupby(['Numero', 'Conto']).sum()

print(grouped)

validated = []

for name, group in grouped.groupby(level=0):
  #print('---')
  boolean = True
  numero_fattura = name
  mask1 = (prima_nota['ND ori.'] == numero_fattura)
  #print(numero_fattura)
  for name1, group in group.groupby(level=[0, 1]):
    conto = name1[1]
    ammontare = group.values[0][0]
    #print(conto)
    #print(ammontare)
    mask2 = (prima_nota['Conto'] == conto)
    if not (prima_nota.Importo[mask1][mask2].empty):
      #print(prima_nota.Importo[mask1][mask2].values[0])
      if (np.abs((prima_nota.Importo[mask1][mask2].values[0] - ammontare)) > 0.01):
        boolean = False
    else:
      boolean = False
  if boolean:
    validated.append(numero_fattura)

print(validated)
    



                       PrezzoTotale
Numero      Conto                  
12/1040     66/25/006       153.310
12/871      66/25/005        82.940
            66/25/006       161.670
18420       66/25/006      1448.510
19048       66/25/006      1671.570
...                             ...
8874/0      66/20/005       151.330
            66/30/015        79.000
            66/30/017        54.600
            66/30/055         0.300
FPR 1025/20 66/25/006        73.962

[146 rows x 1 columns]
['53138', '55496', '56723', '57898', '65226', '71285', '7699/0', '8874/0']


In [17]:
#creazione di un nuovo data_labeled con questi nuovi samples validati dagli ammontari in prima nota

mask = []
for i in list(data_unlabeled3.Numero):
  boolean = False
  if i in validated:
    boolean = True
  mask.append(boolean)

data_labeled3 = data_unlabeled3[mask]
data_labeled3.reset_index(inplace=True)
data_labeled3 = data_labeled3.drop(['index', 'Conto'], axis=1)

y_pred_validated = y_pred[mask]
data_labeled3 = pd.concat(objs=[data_labeled3, pd.Series(y_pred_validated).rename('Conto')], axis=1)

print(data_labeled3.shape)

(152, 1564)


In [18]:
#data_labeled3 contiene solo le pseudo-labels validate subito perchè verosimilmente il modello predittivo non ha commesso nessun
#errore. Però, esplorando lo spazio degli swap con A* se ne possono validare anche molte altre. 

non_validated = set(list(np.unique(data_unlabeled3.Numero))) - set(validated)
print(len(non_validated))

#A* viene quindi applicato per 34 fatture

40


In [14]:
#A* viene applicato su tutte le 48 fatture (validate immediatamente e non) che raggruppano le 1049 righe con delle pseudo-labels.
#Si riesce a determinare la sequenza di swap per validare le pseudo-labels per 38 fatture (per un totale di 747 righe) come riscontrabile dal confronto tra prima 
#nota e raggruppamento per ContoEsatto (cioè il conto assegnato dopo gli swap)

df_Astar_validated = pd.read_excel('/content/pseudo_labels_validated2.xlsx')
grouped2 = df_Astar_validated.groupby(['Numero', 'ContoEsatto']).sum()

print(df_Astar_validated.shape)
print(grouped2)


(1049, 5)
                         Unnamed: 0  PrezzoTotale
Numero      ContoEsatto                          
12/1040     66/25/005            68        76.680
            66/25/006           102        76.630
12/871      66/25/005           508        96.440
            66/25/006          1769       148.170
2069036018  66/25/005          3075       184.670
...                             ...           ...
8874/0      66/20/005          1197       151.330
            66/30/015          1194        79.000
            66/30/017           395        54.600
            66/30/055           402         0.300
FPR 1025/20 66/20/005          2212        73.962

[134 rows x 2 columns]


In [16]:
#valutazione delle performance dello pseudo labeling con validazione:
#in df_Astar_validated, le fatture per cui l'euristica non ha funzionato hanno ContoEsatto = NaN, le rimuovo

df_Astar_validated = df_Astar_validated[df_Astar_validated['ContoEsatto'].notna()]
Astar_fatture = list(np.unique(df_Astar_validated.Numero))

print(df_Astar_validated.shape)
print(len(Astar_fatture))

#Nel complesso si sono validate, grazie all'approccio semi-supervised, 747 righe su 1156 possibili (cioè il 65%).

(747, 5)
38


In [17]:
#adesso le ulteriori pseudo-labels validate da A* devono essere aggiunte ai corrispondenti dati unsupervised di 
#data_unlabeled3 (cioè il dataset di samples unsupervised ripulito), creando un nuovo dataset supervised: data_labeled4

mask = []
for i in list(data_unlabeled3.Numero):
  boolean = False
  if i in Astar_fatture:
    boolean = True
  mask.append(boolean)

data_labeled4 = data_unlabeled3[mask]
data_labeled4.reset_index(inplace=True)
data_labeled4 = data_labeled4.drop(['index', 'Conto'], axis=1)

for numero_fattura in list(np.unique(data_labeled4.Numero)):
  data_labeled4.loc[data_labeled4['Numero'] == numero_fattura, 'Conto'] = df_Astar_validated.loc[df_Astar_validated['Numero'] == numero_fattura, 'ContoEsatto'].values


In [23]:
#creazione di un dataset supervisionato esteso in cui ci sono tutte le coppie input-output ottenute dalla risoluzione
#del knapsack problem (1805, data_labeled), più tutte quelle appena validate (747, data_labeled4)

data_labeled5 = pd.concat(objs = [data_labeled, data_labeled4], axis=0)
data_labeled5.reset_index(inplace=True)
data_labeled5 = data_labeled5.drop(['index'], axis=1)

print(data_labeled5.shape)

print(data_labeled5.groupby('Conto').count())

#In totale si hanno 2552 dati supervisionati, di cui 747 grazie ad A* (un'estensione del 41%)

(2552, 1564)
           Descrizione  Numero  PrezzoUnitario  PrezzoTotale  AliquotaIVA  \
Conto                                                                       
18/40/501            1       1               1             1            1   
66/05/006          790     790             790           790          790   
66/20/005          344     344             344           344          344   
66/25/005          300     300             300           300          300   
66/25/006          516     516             516           516          516   
66/25/505           12      12              12            12           12   
66/25/506           34      34              34            34           34   
66/25/507           22      22              22            22           22   
66/25/508            1       1               1             1            1   
66/25/509           53      53              53            53           53   
66/30/015           48      48              48            48   

In [24]:
#rimozione delle unità statistiche con target troppo rari (threshold = 5, incluso). La molteplicità dei target scende da 24 a 14.

mask = (data_labeled5['Conto'] != '18/40/501')&(data_labeled5['Conto'] != '66/25/508')&(data_labeled5['Conto'] != '66/30/060')&(data_labeled5['Conto'] != '68/05/005')&(data_labeled5['Conto'] != '68/05/133')&(data_labeled5['Conto'] != '68/05/290')&(data_labeled5['Conto'] != '68/05/320')&(data_labeled5['Conto'] != '68/05/385')&(data_labeled5['Conto'] != '68/05/407')&(data_labeled5['Conto'] != '88/20/035')
data_labeled5 = data_labeled5[mask]
data_labeled5.reset_index(inplace=True)
data_labeled5 = data_labeled5.drop(['index'], axis=1)
print(data_labeled5.shape)

#rimangono 2535 dati per training e test

(2535, 1564)


In [25]:
#per rendere le performance confrontabili con il caso base senza estensione del dataset supervisionato grazie alle pseudo-labels
#uso come test set lo stesso del notebook 1, ovvero il 30% finale di data_labeled: cioè le righe dalla 1264 alla 1804 di
#data_labeled5

data_test = data_labeled5.loc[1264:1804,:]

data_train = data_labeled5.drop(data_test.index, axis=0)
data_test.reset_index(inplace=True)
data_test = data_test.drop(['index'], axis=1)
data_train.reset_index(inplace=True)
data_train = data_train.drop(['index'], axis=1)
print(data_test.shape)
print(data_train.shape)

#In definitiva, su 5877 righe fattura iniziali, solo 2961 sono effettivamente associabili a un output in prima nota e la pipeline
#adottata permette di etichettarne 2552, cioè l'86%

(541, 1564)
(1994, 1564)


In [26]:
#test dei soliti modelli RF e AdaBoost sul solito test set, ma dopo il training su data_train esteso grazie ai dati semi supervised

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

features = data_train.drop(['Conto', 'Descrizione', 'Numero'], axis=1)
targets = data_train.Conto

clf2 = RandomForestClassifier(n_estimators=100, max_depth=50, bootstrap = False)
y_pred = clf2.fit(features, targets).predict(data_test.drop(['Conto', 'Numero', 'Descrizione'], axis = 1))
print(accuracy_score(data_test.Conto, y_pred))
print(len(y_pred))

#la performance di accuracy è salita stabilmente sopra al 95%

0.9519408502772643
541


In [27]:
#AdaBoost

from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree

clf2 = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=7), n_estimators=100)
y_pred = clf2.fit(features, targets).predict(data_test.drop(['Conto', 'Numero', 'Descrizione'], axis = 1))
print(accuracy_score(data_test.Conto, y_pred))

#Per AdaBoost la performance non sembra variare (rimane tra 94% e 95%).
#Sia nel caso di RF che di AdaBoost, gli errori nella matrice di confusione sono dello stesso tipo di prima, ma nel caso di RF,
#grazie allo pseudo-labeling si è ottenuto un +1% di accuracy sullo stesso test set.

0.9426987060998152


In [ ]:
#tentativo CatBoost

!pip3 install catboost
import catboost as cb

model_cat_def = cb.CatBoostClassifier(iterations=1000, learning_rate=0.1, max_depth=6)
y_pred = model_cat_def.fit(features, targets).predict(data_test.drop(['Conto', 'Numero', 'Descrizione'], axis = 1))
print(accuracy_score(data_test.Conto, y_pred))

#Il tempo di training varia molto al variare della profondità, ma comunque essendo un metodo di boosting non servono
#alberi molto profondi. Di base implementa il classico Gradient Boosting, con qualche miglioria, e la performance è circa 95%

In [64]:
#dato che a seguito dello pseudo-labeling ottenuto grazie ad A* si ha un nuovo, esteso training set, si può creare un nuovo, esteso
#dizionario per Bag of Words, che tenga in considerazione anche le parole che compaiono nei nuovi samples fornendo una descrizione piu completa.
#Vanno eliminate da data_labeled5 le precedenti features testuali e aggiunte le nuove

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

data_labeled6 = data_labeled5.drop(data_labeled5.columns[[range(206, 1564)]], axis=1)
data_test = data_labeled6.loc[1264:1804,:]
data_train = data_labeled6.drop(data_test.index, axis=0)

descr = data_labeled6.Descrizione
descr_train = descr[data_train.index]
descr_test = descr[data_test.index]

vectorizer = CountVectorizer()
bag_train = vectorizer.fit_transform(list(descr_train))
bag_train = pd.DataFrame(bag_train.toarray(), columns=vectorizer.get_feature_names_out())

bag_test = vectorizer.transform(list(descr_test))
bag_test = pd.DataFrame(bag_test.toarray(), columns=vectorizer.get_feature_names_out())

data_train.reset_index(inplace=True)
data_train = data_train.drop(['index'], axis=1)
data_test.reset_index(inplace=True)
data_test = data_test.drop(['index'], axis=1)
bag_train.reset_index(inplace=True)
bag_train = bag_train.drop(['index'], axis=1)
bag_test.reset_index(inplace=True)
bag_test = bag_test.drop(['index'], axis=1)

data_train = pd.concat(objs=(data_train, bag_train), axis=1)
data_test = pd.concat(objs=(data_test, bag_test), axis=1)

print(data_train.shape)
print(data_test.shape)


(1994, 1905)
(541, 1905)


/usr/local/lib/python3.7/dist-packages/pandas/core/indexes/base.py:4616: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


In [92]:
#RF: con profondità aumentata perchè ora ci sono più features

features = data_train.drop(['Conto', 'Descrizione', 'Numero'], axis=1)
targets = data_train.Conto

clf2 = RandomForestClassifier(n_estimators=100, max_depth=75, bootstrap = False)
y_pred = clf2.fit(features, targets).predict(data_test.drop(['Conto', 'Numero', 'Descrizione'], axis = 1))
print(accuracy_score(data_test.Conto, y_pred))

#usando il solito test set di sempre, l'accuracy è ora quasi del 96%

0.9593345656192237
